In [1]:
import os
import glob
import matplotlib
import numpy as np
import pandas as pd
import multiprocessing
import PersistenceImages.persistence_images as pimg

from ripser import Rips
from matplotlib import cm
from scipy.io import loadmat
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
from matplotlib.colors import ListedColormap

In [2]:
def get_sim_data(timefile, simulationtype):
    
    posfilestring = simulationtype + "/" + "Pos_" + str(timefile) +".dat"
    posdata = np.loadtxt(posfilestring, str, "#", ",")
    posx = []
    posy = []
    for val in posdata:
        splitpos = val.split("+")
        xpos = ""
        ypos = ""
        if(len(splitpos) == 2):
            xpos = splitpos[0]
            ypos = splitpos[1]
        else:
            minusindex = val[1:].find("-") + 1
            xpos = val[0:minusindex]
            ypos = val[minusindex:]
        ypos = ypos[:-1]
        xpos = [float(xpos)]
        ypos = [float(ypos)]
        posx = np.append(posx, xpos)
        posy = np.append(posy, ypos)   
    positions = np.column_stack((posx, posy))
    typefilestring = simulationtype + "/" + "Types_" + str(timefile) + ".dat"
    typedata = np.loadtxt(typefilestring, int, "#", ",")
    return (positions, typedata)

def compute_dim0_img(greenxpos, greenypos, redxpos, redypos, xpos, ypos, max_eps):
    
    vrc = Rips(maxdim = 0, thresh = max_eps)
    green_diag = vrc.fit_transform(np.transpose(np.array([greenxpos, greenypos])))
    red_diag = vrc.fit_transform(np.transpose(np.array([redxpos, redypos])))
    all_diag = vrc.fit_transform(np.transpose(np.array([xpos, ypos])))
    
    green_diag[0][-1] = [0.0, max_eps]
    red_diag[0][-1] = [0.0, max_eps]
    all_diag[0][-1] = [0.0, max_eps]
    
    pers_imager = pimg.PersistenceImager()
    pers_imager.pixel_size = 0.1
    pers_imager.birth_range = (0, 1)
    pers_imager.pers_range = (0, max_eps)
    pers_imager.weight = pimg.weighting_fxns.persistence
    #pers_imager.weight_params = {'n': 1}
    #pers_imager.kernel_params = {'sigma': 1.0}
    #print(pers_imager)
    
    green_pers_img = pers_imager.transform(green_diag[0], skew=False)
    red_pers_img = pers_imager.transform(red_diag[0], skew=False)
    all_pers_img = pers_imager.transform(all_diag[0], skew=False)
    
    return (green_pers_img, red_pers_img, all_pers_img)

def compute_pers_img(greenxpos, greenypos, redxpos, redypos, xpos, ypos, dim, max_eps):
    
    if dim == 0:
        return compute_dim0_img(greenxpos, greenypos, redxpos, redypos, xpos, ypos, max_eps)
    
    vrc = Rips(maxdim = dim, thresh = max_eps)
    green_diag = vrc.fit_transform(np.transpose(np.array([greenxpos, greenypos])))
    red_diag = vrc.fit_transform(np.transpose(np.array([redxpos, redypos])))
    all_diag = vrc.fit_transform(np.transpose(np.array([xpos, ypos])))
    
    pers_imager = pimg.PersistenceImager()
    pers_imager.pixel_size = 0.1
    pers_imager.birth_range = (0, max_eps)
    pers_imager.pers_range = (0, max_eps/2)
    pers_imager.weight = pimg.weighting_fxns.persistence
    #print(pers_imager)
    
    green_pers_img = pers_imager.transform(green_diag[dim], skew=True)
    red_pers_img = pers_imager.transform(red_diag[dim], skew=True)
    all_pers_img = pers_imager.transform(all_diag[dim], skew=True)
    
    return (green_pers_img, red_pers_img, all_pers_img)
    
def visualize(posdata, dim0_imgs, dim1_imgs, dim2_imgs, max_eps, ofolder, ofilename):
    
    greenxpos, greenypos, redxpos, redypos = posdata
    green_img_dim0, red_img_dim0, all_img_dim0 = dim0_imgs
    green_img_dim1, red_img_dim1, all_img_dim1 = dim1_imgs
    green_img_dim2, red_img_dim2, all_img_dim2 = dim2_imgs
    
    green_intensity_vals = np.mean(green_img_dim0, axis=0)
    red_intensity_vals = np.mean(red_img_dim0, axis=0)
    all_intensity_vals = np.mean(all_img_dim0, axis=0)
    
    white = np.array([1.0, 1.0, 1.0, 1.0])
    reds_cmap = cm.get_cmap('Reds', 20)
    redcolors = reds_cmap(range(12))
    redcolors[0, :] = white
    red_cmp = ListedColormap(redcolors)
    greens_cmap = cm.get_cmap('Greens', 20)
    greencolors = greens_cmap(range(12))
    greencolors[0, :] = white
    green_cmp = ListedColormap(greencolors)
        
    xs = range(0, int(max_eps/0.1))
    xs = [x*0.1 for x in xs]
    xs = np.round(xs, decimals=2)
    
    plt.figure(figsize=(12,10), dpi=200)
    plt.subplot(221)
    plt.scatter(greenxpos, greenypos, s=3, color='green')
    plt.scatter(redxpos, redypos, s=3, color='red')
    plt.xticks([])
    plt.yticks([])
    plt.xlim([-10, 10])
    plt.ylim([-10, 10])
    plt.subplot(222)
    plt.plot(xs, green_intensity_vals, color='green')
    plt.plot(xs, red_intensity_vals, color='red')
    plt.plot(xs, all_intensity_vals, color='black')
    plt.xlabel('$H_0$ Interval Length', size=12)
    plt.ylabel('Image Intensity $\propto$ Interval Length', size=12)
    plt.subplot(223)
    plt.imshow(np.flipud(np.transpose(green_img_dim1)), cmap=green_cmp)
    plt.imshow(np.flipud(np.transpose(red_img_dim1)), alpha=0.5, cmap=red_cmp, interpolation='bilinear')
    plt.xlabel('$H_1$ Interval Start', size=12)
    plt.ylabel('$H_1$ Interval Length', size=12)
    plt.subplot(224)
    plt.imshow(np.flipud(np.transpose(green_img_dim2)), cmap=green_cmp)
    plt.imshow(np.flipud(np.transpose(red_img_dim2)), alpha=0.5, cmap=red_cmp, interpolation='bilinear')
    plt.xlabel('$H_2$ Interval Start', size=12)
    plt.ylabel('$H_2$ Interval Length', size=12)
    plt.subplots_adjust(wspace=0.2, hspace=0)
    plt.savefig(ofolder + os.sep + ofilename)
    plt.close()
    #plt.show()

In [3]:
foldername = "DAH_Lipid_Bilayer_1"
output_folder = foldername + "_persimg"

if not os.path.isdir(output_folder):
    os.mkdir(output_folder)
    
pos_file_list = glob.glob(foldername + os.sep + "Pos_*.dat")
frame_values = []
for pos_file in pos_file_list:
    fname = pos_file.split(os.sep)[1]
    bname = fname.split(".")[0]
    frame_values.append(int(bname.split("_")[1]))

In [4]:
max_eps = 20

for frame in frame_values:

    (cell_pos, cell_type) = get_sim_data(frame, foldername)

    xpos = cell_pos[:,0]
    ypos = cell_pos[:,1]
    greens = cell_type == 1
    reds = cell_type == 2
    greenxpos = xpos[greens]
    greenypos = ypos[greens]
    redxpos = xpos[reds]
    redypos = ypos[reds]
    
    print("Processing frame: " + repr(frame))

    
    (green_img_dim0, red_img_dim0, all_img_dim0) = compute_pers_img(greenxpos, greenypos, redxpos, redypos, 
                                                                    xpos, ypos, 0, max_eps)

    (green_img_dim1, red_img_dim1, all_img_dim1) = compute_pers_img(greenxpos, greenypos, redxpos, redypos, 
                                                                    xpos, ypos, 1, max_eps)

    (green_img_dim2, red_img_dim2, all_img_dim2) = compute_pers_img(greenxpos, greenypos, redxpos, redypos, 
                                                                    xpos, ypos, 2, max_eps)

    visualize([greenxpos, greenypos, redxpos, redypos], 
              [green_img_dim0, red_img_dim0, all_img_dim0], 
              [green_img_dim1, red_img_dim1, all_img_dim1], 
              [green_img_dim2, red_img_dim2, all_img_dim2],
              max_eps, output_folder, repr(frame) + ".png")

Processing frame: 1000000
Rips(maxdim=0, thresh=20, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
Rips(maxdim=1, thresh=20, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
Rips(maxdim=2, thresh=20, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
